In [0]:
USE CATALOG users;
USE users.yukiteru_koide;

-- 1) 日次サマリー
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_daily_summary AS
SELECT
  event_date,
  COUNT(*) AS total_signins,
  SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) AS failure_count,
  COUNT(DISTINCT userId) AS unique_users
FROM users.yukiteru_koide.signin_events_silver
GROUP BY event_date;

-- 2) 失敗理由 × 日
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_fail_by_reason_day AS
SELECT
  event_date,
  coalesce(statusFailureReason, 'Unknown') AS failure_reason,
  COUNT(*) AS failures
FROM users.yukiteru_koide.signin_events_silver
WHERE is_failure
GROUP BY event_date, coalesce(statusFailureReason, 'Unknown');

-- 3) アプリ別 × 日
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_by_app_day AS
SELECT
  event_date,
  appDisplayName,
  SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS success_count,
  SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) AS failure_count
FROM users.yukiteru_koide.signin_events_silver
GROUP BY event_date, appDisplayName;

-- 4) Impossible Travel 明細
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_geo_impossible_travel AS
SELECT
  created_ts, userPrincipalName, userId,
  locationCountryOrRegion, locationCity,
  prev_country, prev_city,
  impossible_travel,
  ipAddress, clientAppUsed, deviceOperatingSystem, browser
FROM users.yukiteru_koide.signin_events_silver
WHERE impossible_travel = true;

-- 5) ユーザー別リスク（日次）
CREATE OR REPLACE TABLE users.yukiteru_koide.signin_risk_by_user_day AS
SELECT
  userId,
  ANY_VALUE(userPrincipalName) AS userPrincipalName,
  event_date,
  SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) AS failed_logins,
  SUM(CASE WHEN is_off_hours AND is_success THEN 1 ELSE 0 END) AS offhours_success,
  SUM(CASE WHEN ca_failed THEN 1 ELSE 0 END) AS ca_failures,
  SUM(CASE WHEN impossible_travel THEN 1 ELSE 0 END) AS impossible_travels,
  (SUM(CASE WHEN is_failure THEN 1 ELSE 0 END) * 1)
  + (SUM(CASE WHEN is_off_hours AND is_success THEN 1 ELSE 0 END) * 2)
  + (SUM(CASE WHEN ca_failed THEN 1 ELSE 0 END) * 2)
  + (SUM(CASE WHEN impossible_travel THEN 1 ELSE 0 END) * 5) AS risk_score
FROM users.yukiteru_koide.signin_events_silver
GROUP BY userId, event_date;

-- ダッシュボード用ビュー
CREATE OR REPLACE VIEW users.yukiteru_koide.v_dashboard_cards AS
SELECT
  (SELECT SUM(success_count) FROM users.yukiteru_koide.signin_daily_summary
   WHERE event_date >= date_sub(current_date(), 7)) AS success_7d,
  (SELECT SUM(failure_count) FROM users.yukiteru_koide.signin_daily_summary
   WHERE event_date >= date_sub(current_date(), 7)) AS failure_7d,
  (SELECT COUNT(*) FROM users.yukiteru_koide.signin_geo_impossible_travel
   WHERE created_ts >= date_sub(current_date(), 7)) AS impossible_travel_7d;

OPTIMIZE users.yukiteru_koide.signin_daily_summary;
OPTIMIZE users.yukiteru_koide.signin_fail_by_reason_day;
OPTIMIZE users.yukiteru_koide.signin_by_app_day;
OPTIMIZE users.yukiteru_koide.signin_geo_impossible_travel;
OPTIMIZE users.yukiteru_koide.signin_risk_by_user_day;
